In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/correct/capstone_dataset_hugging_face.csv
/kaggle/input/accuracy94/phoneme_correction_model_editDistance_accuracy94.h5


In [2]:
import pandas as pd

# Load the CSV file
file_path = '/kaggle/input/correct/capstone_dataset_hugging_face.csv'  # Replace with the path to your CSV file
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())


     word ipa_phoneme incorrect_ipa_phoneme
0      aa         ˈɑː                    ˈː
1     aah         ˈɑː                    ˈɑ
2   aahed        ˈɑːd                   ɑːd
3  aahing       ˈɑːɪŋ                  ˈːɪŋ
4    aahs        ˈɑːz                  ˈθːz


In [4]:
# Save the entire model
model.save('phoneme_correction_model_editDistance.h5')


In [4]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('/kaggle/input/accuracy94/phoneme_correction_model_editDistance_accuracy94.h5')


In [11]:
new_mispronounced_sequence = "ɐbdˈʌks"  # Replace with actual sequence
new_mispronounced_sequence_tokenized = tokenizer.texts_to_sequences([new_mispronounced_sequence])
new_mispronounced_sequence_padded = pad_sequences(new_mispronounced_sequence_tokenized, maxlen=max_sequence_length, padding='post')

# Prepare the decoder input for inference
decoder_input = np.zeros_like(new_mispronounced_sequence_padded)
decoder_input[:, 1:] = new_mispronounced_sequence_padded[:, :-1]

# Predict
predicted_phonemes = loaded_model.predict([new_mispronounced_sequence_padded, decoder_input])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [ ]:
#94 accuracy - SEQ2SEQ WITH LSTM LAYERS 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
import numpy as np

# Load the CSV file
file_path = '/kaggle/input/correct/capstone_dataset_hugging_face.csv'  # Replace with the path to your CSV file
df = pd.read_csv(file_path)

# Combine all phoneme sequences to fit the tokenizer
all_phonemes = list(df['ipa_phoneme']) + list(df['incorrect_ipa_phoneme'])

tokenizer = Tokenizer(char_level=True)  # Use character-level tokenization
tokenizer.fit_on_texts(all_phonemes)

# Create tokenized sequences
correct_sequences = tokenizer.texts_to_sequences(df['ipa_phoneme'])
incorrect_sequences = tokenizer.texts_to_sequences(df['incorrect_ipa_phoneme'])

# Pad sequences to the same length
max_sequence_length = max(max(len(seq) for seq in correct_sequences), max(len(seq) for seq in incorrect_sequences))

correct_sequences = pad_sequences(correct_sequences, maxlen=max_sequence_length, padding='post')
incorrect_sequences = pad_sequences(incorrect_sequences, maxlen=max_sequence_length, padding='post')

2024-09-25 15:03:44.354857: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 15:03:44.355000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 15:03:44.477927: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:


# Split the data into training and validation sets
train_incorrect, val_incorrect, train_correct, val_correct = train_test_split(
    incorrect_sequences, correct_sequences, test_size=0.2, random_state=42
)

# Prepare decoder input sequences for training (shifted incorrect sequences)
train_decoder_input_sequences = np.zeros_like(train_incorrect)
train_decoder_input_sequences[:, 1:] = train_incorrect[:, :-1]

val_decoder_input_sequences = np.zeros_like(val_incorrect)
val_decoder_input_sequences[:, 1:] = val_incorrect[:, :-1]

# Prepare target sequences for training (correct sequences)
train_decoder_target_sequences = np.expand_dims(train_correct, -1)
val_decoder_target_sequences = np.expand_dims(val_correct, -1)

# Define model parameters
embedding_dim = 64
latent_dim = 128
num_tokens = len(tokenizer.word_index) + 1  # +1 for padding token

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_tokens, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_tokens, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Seq2Seq Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    [train_incorrect, train_decoder_input_sequences],
    train_decoder_target_sequences,
    epochs=50,
    batch_size=64,
    validation_data=([val_incorrect, val_decoder_input_sequences], val_decoder_target_sequences)
)

# Evaluate the model on the validation data
val_loss, val_accuracy = model.evaluate([val_incorrect, val_decoder_input_sequences], val_decoder_target_sequences)
print(f"Validation Accuracy: {val_accuracy:.4f}")


Epoch 1/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 33s 9ms/step - accuracy: 0.7655 - loss: 0.8888 - val_accuracy: 0.8471 - val_loss: 0.5193
Epoch 2/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - accuracy: 0.8603 - loss: 0.4693 - val_accuracy: 0.8897 - val_loss: 0.3611
Epoch 3/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - accuracy: 0.8965 - loss: 0.3375 - val_accuracy: 0.9076 - val_loss: 0.3009
Epoch 4/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - accuracy: 0.9122 - loss: 0.2833 - val_accuracy: 0.9181 - val_loss: 0.2639
Epoch 5/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - accuracy: 0.9205 - loss: 0.2548 - val_accuracy: 0.9229 - val_loss: 0.2463
Epoch 6/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - accuracy: 0.9263 - loss: 0.2353 - val_accuracy: 0.9267 - val_loss: 0.2335
Epoch 7/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - accuracy: 0.9311 - loss: 0.2196 - val_accuracy: 0.9303 - val_loss: 0.2226
Epoch 8/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - accuracy: 0.9350 - loss: 0

In [14]:
model.save('phoneme_correction_model_editDistance_accuracy94.h5')

In [4]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('/kaggle/input/accuracy94/phoneme_correction_model_editDistance_accuracy94.h5')


In [3]:
new_mispronounced_sequence = ""  # Replace with actual sequence
new_mispronounced_sequence_tokenized = tokenizer.texts_to_sequences([new_mispronounced_sequence])
new_mispronounced_sequence_padded = pad_sequences(new_mispronounced_sequence_tokenized, maxlen=max_sequence_length, padding='post')

# Prepare the decoder input for inference
decoder_input = np.zeros_like(new_mispronounced_sequence_padded)
decoder_input[:, 1:] = new_mispronounced_sequence_padded[:, :-1]

# Predict
predicted_phonemes = loaded_model.predict([new_mispronounced_sequence_padded, decoder_input])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [4]:
# Get the token with the highest probability for each timestep
predicted_token_indices = np.argmax(predicted_phonemes, axis=-1)

# Create a reverse mapping from index to phoneme
index_to_phoneme = {index: phoneme for phoneme, index in tokenizer.word_index.items()}

# Add the padding token to the reverse mapping
index_to_phoneme[0] = ''  # Assuming 0 is the padding token

# Decode the predicted token indices to phoneme sequences
predicted_phoneme_sequences = []
for token_sequence in predicted_token_indices:
    predicted_phoneme_sequence = ''.join([index_to_phoneme[token] for token in token_sequence])
    predicted_phoneme_sequences.append(predicted_phoneme_sequence)

# Print the decoded phoneme sequence
for seq in predicted_phoneme_sequences:
    print(seq)



plˈeɪɡ


In [3]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the model
loaded_model = load_model('/kaggle/input/accuracy94/phoneme_correction_model_editDistance_accuracy94.h5')

# List of new mispronounced sequences
new_mispronounced_sequences = [
  "kˈæməflˌɑːʒɛ"

]

# Function to predict and decode the sequence
def predict_and_decode(sequence):
    # Tokenize and pad the sequence
    tokenized_sequence = tokenizer.texts_to_sequences([sequence])
    padded_sequence = pad_sequences(tokenized_sequence, maxlen=max_sequence_length, padding='post')

    # Prepare the decoder input for inference
    decoder_input = np.zeros_like(padded_sequence)
    decoder_input[:, 1:] = padded_sequence[:, :-1]

    # Predict the phonemes
    predicted_phonemes = loaded_model.predict([padded_sequence, decoder_input])
    
    # Get the token with the highest probability for each timestep
    predicted_token_indices = np.argmax(predicted_phonemes, axis=-1)

    # Create a reverse mapping from index to phoneme
    index_to_phoneme = {index: phoneme for phoneme, index in tokenizer.word_index.items()}
    index_to_phoneme[0] = ''  # Assuming 0 is the padding token

    # Decode the predicted token indices to phoneme sequence
    predicted_phoneme_sequence = ''.join([index_to_phoneme[token] for token in predicted_token_indices[0]])

    return predicted_phoneme_sequence

# Iterate over the list of mispronounced sequences and get the corrected output for each
corrected_phoneme_sequences = [predict_and_decode(seq) for seq in new_mispronounced_sequences]

# Print the decoded phoneme sequences
for original, corrected in zip(new_mispronounced_sequences, corrected_phoneme_sequences):
    print(f"Original: {original} -> Corrected: {corrected}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
Original: kˈæməflˌɑːʒɛ -> Corrected: kˈæməllˌɑːd


In [ ]:
# phoneme correction with BILSTM - accuracy 95%

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, TimeDistributed
import numpy as np

# Load the CSV file
file_path = '/kaggle/input/correct/capstone_dataset_hugging_face.csv'  # Replace with the path to your CSV file
df = pd.read_csv(file_path)

# Combine all phoneme sequences to fit the tokenizer
all_phonemes = list(df['ipa_phoneme']) + list(df['incorrect_ipa_phoneme'])

tokenizer = Tokenizer(char_level=True)  # Use character-level tokenization
tokenizer.fit_on_texts(all_phonemes)

# Create tokenized sequences
correct_sequences = tokenizer.texts_to_sequences(df['ipa_phoneme'])
incorrect_sequences = tokenizer.texts_to_sequences(df['incorrect_ipa_phoneme'])

# Pad sequences to the same length
max_sequence_length = max(max(len(seq) for seq in correct_sequences), max(len(seq) for seq in incorrect_sequences))

correct_sequences = pad_sequences(correct_sequences, maxlen=max_sequence_length, padding='post')
incorrect_sequences = pad_sequences(incorrect_sequences, maxlen=max_sequence_length, padding='post')

In [1]:


# Split the data into training and validation sets
train_incorrect, val_incorrect, train_correct, val_correct = train_test_split(
    incorrect_sequences, correct_sequences, test_size=0.2, random_state=42
)

# Define model parameters
embedding_dim = 64
latent_dim = 128
num_tokens = len(tokenizer.word_index) + 1  # +1 for padding token

# Input layer for incorrect phoneme sequences
inputs = Input(shape=(max_sequence_length,))

# Embedding layer
embedding_layer = Embedding(num_tokens, embedding_dim,
input_length=max_sequence_length)(inputs)

# BiLSTM layer
bilstm_layer = Bidirectional(LSTM(latent_dim, return_sequences=True))
(embedding_layer)

# TimeDistributed Dense layer to output a corrected phoneme at each time step
output_layer = TimeDistributed(Dense(num_tokens, activation='softmax'))
(bilstm_layer)

# Define the model
model = Model(inputs, output_layer)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Expand the correct sequences to match the expected output shape
train_correct_expanded = np.expand_dims(train_correct, -1)
val_correct_expanded = np.expand_dims(val_correct, -1)

# Train the model
history = model.fit(
    train_incorrect,
    train_correct_expanded,
    epochs=50,
    batch_size=64,
    validation_data=(val_incorrect, val_correct_expanded)
)

# Evaluate the model on the validation data
val_loss, val_accuracy = model.evaluate(val_incorrect, val_correct_expanded)
print(f"Validation Accuracy: {val_accuracy:.4f}")


2024-09-13 04:27:30.994905: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-13 04:27:30.995014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-13 04:27:31.118819: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 50s 12ms/step - accuracy: 0.8585 - loss: 0.5285 - val_accuracy: 0.9284 - val_loss: 0.2134
Epoch 2/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 42s 12ms/step - accuracy: 0.9332 - loss: 0.2002 - val_accuracy: 0.9402 - val_loss: 0.1771
Epoch 3/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 41s 12ms/step - accuracy: 0.9447 - loss: 0.1672 - val_accuracy: 0.9461 - val_loss: 0.1621
Epoch 4/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 41s 12ms/step - accuracy: 0.9490 - loss: 0.1533 - val_accuracy: 0.9486 - val_loss: 0.1548
Epoch 5/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 82s 12ms/step - accuracy: 0.9519 - loss: 0.1445 - val_accuracy: 0.9495 - val_loss: 0.1516
Epoch 6/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 41s 12ms/step - accuracy: 0.9540 - loss: 0.1381 - val_accuracy: 0.9510 - val_loss: 0.1476
Epoch 7/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 41s 12ms/step - accuracy: 0.9555 - loss: 0.1334 - val_accuracy: 0.9520 - val_loss: 0.1453
Epoch 8/50
3437/3437 ━━━━━━━━━━━━━━━━━━━━ 41s 12ms/step - accuracy: 0.9568 -

In [3]:
model.save('phoneme_correction_bilstm_model.h5')

In [1]:
from tensorflow.keras.models import load_model

# Load the model from the saved .h5 file
model = load_model('/kaggle/input/bilstm-correction-model/phoneme_correction_bilstm_model.h5')


2024-09-26 03:02:27.713816: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 03:02:27.713975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 03:02:27.872486: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np



# Example phoneme input (replace this with the phoneme sequence you want to test)
input_phoneme_sequence = 'nŋˈɑːvəl'

# Tokenize the phoneme sequence
input_sequence = tokenizer.texts_to_sequences([input_phoneme_sequence])

# Pad the sequence to the maximum length used during training
input_sequence_padded = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')

# Make a prediction
predicted_sequence = model.predict(input_sequence_padded)

# Get the predicted token for each time step (output is probabilities, so we take argmax)
predicted_token_ids = np.argmax(predicted_sequence, axis=-1)

# Convert token IDs back to phonemes
predicted_phonemes = [tokenizer.index_word[token] for token in predicted_token_ids[0] if token > 0]

# Join the predicted phonemes into a readable string
predicted_phoneme_sequence = ' '.join(predicted_phonemes)

# Output the predicted phoneme sequence
print("Predicted corrected phoneme sequence:", predicted_phoneme_sequence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted corrected phoneme sequence: n ˈ ɑ ː v ə l
